<h4 align="right">by <a href="http://cse.iitkgp.ac.in/~adas/">Abir Das</a> with help of <br> Ram Rakesh and Ankit Singh<br> </h4>

### Write the following details here
** Name: ** `DEV NARAYAN YADAV`<br/>
** Roll Number: ** `18CS92R03`<br/>
** Department: ** `COMPUTER SCIENCE AND ENGINEERING`<br/>
** Email: ** `devyadav15294@gmail.com`

# Problem Set 2

## Preamble

To run and solve this assignment, one must have a working IPython Notebook installation. The easiest way to set it up for both Windows and Linux is to install [Anaconda](https://www.continuum.io/downloads). Then save this file ([`assignment_01.ipynb`]()) to your computer, run Anaconda and choose this file in Anaconda's file explorer. Use `Python 3` version. Below statements assume that you have already followed these instructions. If you are new to Python or its scientific library, Numpy, there are some nice tutorials [here](https://www.learnpython.org/) and [here](http://www.scipy-lectures.org/).

### Problem: You will implement a fully connected neural network from scratch in this problem
We marked places where you are expected to add/change your own code with **`##### write your code below #####`** comment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
'''You are not supposed to import any other python library to work on this assignments.'''

2024-01-12 15:12:54.946198: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 15:12:54.948590: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-12 15:12:54.985198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 15:12:54.985228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 15:12:54.986230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

'You are not supposed to import any other python library to work on this assignments.'

In [2]:
'''data is loaded from data directory.
please don't remove the folder '''
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.load('./X_train.npy')
x_train = x_train.flatten().reshape(-1,28*28)
x_train = x_train / 255.0
gt_indices = y_train
train_length = len(x_train)
print("Number of training examples: {:d}".format(train_length))
print("Training Data Shape (X):",x_train.shape)

Number of training examples: 60000
Training Data Shape (X): (60000, 784)


In [3]:
'''Dimensions to be used for creating your model'''

batch_size = 64  # batch size
input_dim = 784  # input dimension
hidden_1_dim = 512  # hidden layer 1 dimension
hidden_2_dim = 256  # hidden layer 2 dimension
output_dim = 10   # output dimension

'''Other hyperparameters'''
learning_rate = 1e-5

In [4]:
#creating one hot vector representation of output classification
y_train = np.zeros((train_length, output_dim))
# print(y.shape, gt_indices.shape)
for i in range(train_length):
    y_train[i,gt_indices[i]] = 1

# Number of mini-batches (as integer) in one epoch
num_minibatches = np.floor(train_length/batch_size).astype(int)
print("Training Data Shape (Y):",y_train.shape)

Training Data Shape (Y): (60000, 10)


In [5]:
print("No of mini-batches {:d} and total training data used in training:\
{}.".format(num_minibatches, num_minibatches*batch_size))

No of mini-batches 937 and total training data used in training:59968.


In [6]:
'''Randomly Initialize Weights  from standard normal distribution (i.e., mean = 0 and s.d. = 1.0).
Use the dimesnions specified in the cell 3 to initialize your weights matrices. 
Use the nomenclature W1,W2 etc. (provided below) for the different weight matrices.'''

########################## write your code below ##############################################
W1 = np.random.normal(0.0,1.0,(input_dim,hidden_1_dim))
W2 = np.random.normal(0.0,1.0,(hidden_1_dim,hidden_2_dim))
W3 = np.random.normal(0.0,1.0,(hidden_2_dim,output_dim))
###############################################################################################

In [7]:
# Write a function which computes the softmax where X is vector of scores computed during forward pass
def softmax(x):
    ##############################write your code here #################################
    ab = []
    for i in range(0, len(x)):
        a = x[i]
        shift_x = a-np.max(a)
        ab.append(np.exp(shift_x)/np.sum(np.exp(shift_x)))
    ab = np.asarray(ab)
    return ab
    ####################################################################################
#x = [1, 2, 0.5, 3, 0.8]
#print(softmax(x))

In [8]:
def compute_act(weight, train_data,batch_len, batch_wid):
    a = []
    for i in range(0, batch_len):
        ab = np.matmul(weight.T,train_data[i])
        a.append(ab)
    a = np.asarray(a)
    return a

def grad_softmax(pred, onehot,gt_index,b_size):
    grad_soft = np.sum(np.diag(np.take((pred - onehot),gt_index, axis=1)))/b_size
    return grad_soft

def grad_W_3(h2, grad_sm, W3):
    grad = np.sum((grad_sm * h2.T).T, axis = 0) * np.ones((W3.shape[1], 1))
    grad = grad.T
    return grad

def grad_hidd2(pred,onehot,W,batch_len):
    grad = []
    for i in range(0, batch_len):
        ab = np.matmul((pred - onehot)[i], W.T)
        grad.append(ab)
    grad = np.asarray(grad)
    return grad

def mask(a):
    np.place(a, a<0, 0)
    return a

def relu(x):
    return np.maximum(x,0)

def grad_W(h1,hidden_1,grad_a,hidden_2,batch_len):   
    grad = np.sum(np.array([np.matmul(h1[i].reshape((hidden_1, 1)), grad_a[i].reshape((1, hidden_2))) for i in range(batch_size)]), axis=0)/batch_size
    return grad

def grad_hidd1(grad_a,W,batch_len):
    grad = []
    for i in range(0, batch_len):
        ab = np.matmul(grad_a[i], W.T)
        grad.append(ab)
    grad = np.asarray(grad)
    return grad




In [9]:
def train(x_train, y_train, no_of_iterations, learning_rate):
    loss_list=[]
    i_epoch = 0
    W1 = np.random.normal(0.0,1.0,(input_dim,hidden_1_dim))
    W2 = np.random.normal(0.0,1.0,(hidden_1_dim,hidden_2_dim))
    W3 = np.random.normal(0.0,1.0,(hidden_2_dim,output_dim))
    for i_iter in range(no_of_iterations):
    
        ''''''
        # getting one batch
        batch_elem_idx = i_iter%num_minibatches
        x_batchinput = x_train[batch_elem_idx*batch_size:(batch_elem_idx+1)*batch_size]
        y_batchinput = y_train[batch_elem_idx*batch_size:(batch_elem_idx+1)*batch_size]
        y_indices = gt_indices[batch_elem_idx*batch_size:(batch_elem_idx+1)*batch_size]

        ############################## feed forward ##################################
        # first hidden layer implementation
        a1 = compute_act(W1,x_batchinput,batch_size,hidden_1_dim)
        #print("Shape (a1): ",a1.shape)
        # implement Relu layer
        h1 = relu(a1)
        #  implement 2 hidden layer
        a2 = compute_act(W2,h1,batch_size,hidden_2_dim)
        #print("Shape (a2): ",a2.shape)
        # implement Relu activation 
        h2 = relu(a2)
        #implement linear output layer
        a3 = compute_act(W3,h2,batch_size,output_dim)
        #print("Shape (a3): ",a3.shape)
        
        
        # softmax layer
        softmax_score = softmax(a3) #enusre you have implemented the softmax function defined above
        #print("Shape (softmax): ",softmax_score.shape)
    
        ##################################################################################
        ###############################################################################################
    
        neg_log_softmax_score = -np.log(softmax_score+0.00000001) # The small number is added to avoid 0 input to log function
        #print("Shape loss = -log(softmax_score): ",neg_log_softmax_score.shape)
        
        # Compute and print loss
        if i_iter%num_minibatches == 0:
            loss = np.mean(np.diag(np.take(neg_log_softmax_score, gt_indices[batch_elem_idx*batch_size:(batch_elem_idx+1)*batch_size],\
                                           axis=1)))
            print(" Epoch: {:d}, iteration: {:d}, Loss: {:6.4f} ".format(i_epoch, i_iter, loss))
            loss_list.append(loss)
            i_epoch += 1
            # Each 10th epoch reduce learning rate by a factor of 10
            if i_epoch%10 == 0:
                learning_rate /= 10.0
        
        ################################### Backpropagation #####################################
        # Gradient of cross-entropy loss w.r.t. preactivation of the output layer
        grad_softmax_score = grad_softmax(softmax_score, y_batchinput, y_indices, batch_size)
        # gradient w.r.t W3
        grad_W3 = grad_W_3(h2,grad_softmax_score,W3)
        # gradient w.r.t h2
        grad_h2 = grad_hidd2(softmax_score,y_batchinput,W3,batch_size)
        # gradient w.r.t a2
        grad_a2 = grad_h2 * mask(a2)
        # gradient w.r.t W2
        grad_W2 = grad_W(h1,hidden_1_dim,grad_a2,hidden_2_dim,batch_size)
        # gradient w.r.t h1
        grad_h1 = grad_hidd1(grad_a2,W2,batch_size)
        # gradient w.r.t a1
        grad_a1 = grad_h1 * mask(a1)
        # gradient w.r.t W1
        grad_W1 = grad_W(x_batchinput,input_dim,grad_a1,hidden_1_dim,batch_size)
        ####################################################################################################
        
        ################################ Update Weights Block using SGD ####################################
        W3 -= learning_rate * grad_W3
        W2 -= learning_rate * grad_W2
        W1 -= learning_rate * grad_W1
        ####################################################################################################
    return loss_list

In [ ]:
loss_list = train(x_train, y_train, 20000, learning_rate)
#plotting the loss
plt.figure(figsize=(10,5))
plt.plot(loss_list)
plt.title('Loss vs epochs')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.show()

 Epoch: 0, iteration: 0, Loss: 16.9816 
 Epoch: 1, iteration: 937, Loss: 1.4391 
 Epoch: 2, iteration: 1874, Loss: 1.1435 
 Epoch: 3, iteration: 2811, Loss: 1.4391 
 Epoch: 4, iteration: 3748, Loss: 0.8635 
 Epoch: 5, iteration: 4685, Loss: 0.5758 
 Epoch: 6, iteration: 5622, Loss: 0.5756 


In [ ]:
'''Loading the test data from data/X_test.npy and data/y_test.npy.'''
x_test = np.load('./data/X_test.npy')
x_test = x_test.flatten().reshape(-1,28*28)
x_test = x_test / 255.0
y_test = np.load('./data/y_test.npy')

In [ ]:
batch_size_test = 100 # Deliberately taken 100 so that it divides the test data size
num_minibatches = len(y_test)/batch_size_test
test_correct = 0

'''Only forward block code and compute softmax_score .'''
for i_iter in range(int(num_minibatches)):
    
    '''Get one minibatch'''
    batch_elem_idx = i_iter%num_minibatches
    x_batchinput = x_test[i_iter*batch_size_test:(i_iter+1)*batch_size_test]
    
    ######### copy only the forward pass block of your code and pass the x_batchinput to it and compute softmax_score ##########
    # first hidden layer implementation
    a1 = compute_act(W1,x_batchinput,batch_size_test,hidden_1_dim)
    
    # implement Relu layer
    h1 = np.maximum(a1,0)
    
    #  implement 2nd hidden layer
    a2 = compute_act(W2,h1,batch_size_test,hidden_1_dim)
    
    # implement Relu activation 
    h2 = np.maximum(a2,0)
    
    #implement linear output layer
    a3 = compute_act(W3,h2,batch_size_test,hidden_1_dim)
    
    # softmax layer
    softmax_score = softmax(a3) #enusre you have implemented the softmax function defined above
    ##################################################################################
    
    y_batchinput = y_test[i_iter*batch_size_test:(i_iter+1)*batch_size_test]
    
    y_pred = np.argmax(softmax_score, axis=1)
    num_correct_i_iter = np.sum(y_pred == y_batchinput)
    test_correct += num_correct_i_iter
print ("Test accuracy is {:4.2f} %".format(test_correct/len(y_test)*100))